In [1]:
import pandas as pd
import os
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np
import seaborn as sns
from statsmodels.formula.api import ols
import statsmodels.api as sm
from sklearn.linear_model import LogisticRegression

In [2]:
df_all = pd.read_csv('fanfic_regression_data_curated.tsv', sep = '\t')

In [3]:
len(df_all)

698002

In [4]:
df_all['ChapterIndex'].fillna(0, inplace=True)

In [5]:
df_all['Term_novelty'].head()

0    0.888159
1    0.353749
2    0.855796
3    0.706213
4    0.732386
Name: Term_novelty, dtype: float64

In [6]:
df_all['Term_novelty_squared'] = np.square(df_all['Term_novelty'])
df_all['Topic_novelty_squared'] = np.square(df_all['Topic_novelty'])

In [7]:
df_all['Term_novelty_squared'].head(2)

0    0.788826
1    0.125138
Name: Term_novelty_squared, dtype: float64

### Logistic

In [8]:
def run_logit(df, field):
    df = df.replace([np.inf, -np.inf], np.nan)
    df = df.dropna(how = 'any')
    df['Intercept'] = 1.0
    df['binary'] = df[field].apply(lambda x: 1 if x != 0.0 else 0)
    logit_model = sm.Logit(df['binary'], df[['Chapters',\
        'Term_novelty', 'Topic_novelty', 'Term_novelty_squared', 'Topic_novelty_squared',\
        'Freq_relationship', 'Category_F_M',\
       'Category_Gen', 'Category_M_M', 'Category_Multi', 'Category_Other','Category_Unknown', \
       'ArchiveWarnings_underage', 'ArchiveWarnings_death',\
       'ArchiveWarnings_choose_no', 'ArchiveWarnings_no_apply',\
       'ArchiveWarnings_violence',\
       'ArchiveWarnings_noncon', 'author_fic_cnt', 'Rating_G',\
       'Rating_M', 'Rating_N', 'Rating_T',\
       'Fandom_dcu', 'Fandom_doctor_who', 'Fandom_star_wars',
       'Fandom_arthurian', 'Fandom_supernatural', 'Fandom_haikyuu',
       'Fandom_kuroko_no_basuke', 'Fandom_hamilton_miranda',
       'Fandom_dragon_age', 'Fandom_the_walking_dead', 'Fandom_buffy',
       'Fandom_les_miserables', 'Fandom_naruto', 'Fandom_tolkien',
       'Fandom_shakespare', 'Fandom_hetalia', 'Fandom_attack_on_titan',
       'Fandom_ms_paint_adventures', 'Fandom_homestuck', 'Fandom_marvel',
       'Fandom_sailor_moon', 'Fandom_one_direction', 'Fandom_sherlock',
       'History', 'Intercept']]).fit(method='bfgs')
    pred = logit_model.predict()
    df['nonzero_prob'] = pred
    return df

### OLS

In [15]:
def run_ols(df, field):
    df = run_logit(df, field)
    df['Kudos'] = np.log(df['Kudos']+0.001)
    df['Hits'] = np.log(df['Hits']+0.001)
    df['Comments'] = np.log(df['Comments']+1)
    df['Bookmarks'] = np.log(df['Bookmarks']+0.001)
    df = df.replace([np.inf, -np.inf], np.nan)
    df = df.dropna(how = 'any')
    model = ols( field + " ~ Term_novelty + Topic_novelty + Term_novelty_squared + Topic_novelty_squared\
 + Chapters + Freq_relationship + author_fic_cnt + History\
 + Category_Gen + Category_F_M + Category_M_M + Category_Multi + Category_Other + Category_Unknown \
 + ArchiveWarnings_underage + ArchiveWarnings_death + ArchiveWarnings_violence +\
 ArchiveWarnings_noncon + ArchiveWarnings_choose_no +\
 ArchiveWarnings_no_apply + Rating_N + Rating_E + Rating_M + Rating_T\
 + Fandom_dcu + Fandom_doctor_who + Fandom_star_wars + Fandom_arthurian + \
 Fandom_supernatural + Fandom_haikyuu + Fandom_kuroko_no_basuke + Fandom_hamilton_miranda\
 + Fandom_dragon_age + Fandom_the_walking_dead + Fandom_buffy + Fandom_les_miserables \
 + Fandom_naruto + Fandom_tolkien + Fandom_shakespare + Fandom_hetalia + \
 Fandom_attack_on_titan + Fandom_ms_paint_adventures + Fandom_homestuck +\
 Fandom_marvel + Fandom_sailor_moon + Fandom_one_direction + Fandom_sherlock \
 + nonzero_prob", data = df).fit()
    return model

### Plot coefficients

In [16]:
kudos_model = run_ols(df_all, 'Kudos')
kudos_coef = np.asarray(list(kudos_model.params)[1:-1][::-1])
kudos_err = list(kudos_model.bse)[1:-1]

/anaconda3/envs/python36/lib/python3.6/site-packages/statsmodels/discrete/discrete_model.py:1674: RuntimeWarning: overflow encountered in exp
  return 1/(1+np.exp(-X))
/anaconda3/envs/python36/lib/python3.6/site-packages/statsmodels/discrete/discrete_model.py:1724: RuntimeWarning: divide by zero encountered in log
  return np.sum(np.log(self.cdf(q*np.dot(X,params))))
/anaconda3/envs/python36/lib/python3.6/site-packages/statsmodels/discrete/discrete_model.py:1674: RuntimeWarning: overflow encountered in exp
  return 1/(1+np.exp(-X))
/anaconda3/envs/python36/lib/python3.6/site-packages/statsmodels/discrete/discrete_model.py:1724: RuntimeWarning: divide by zero encountered in log
  return np.sum(np.log(self.cdf(q*np.dot(X,params))))
/anaconda3/envs/python36/lib/python3.6/site-packages/statsmodels/discrete/discrete_model.py:1674: RuntimeWarning: overflow encountered in exp
  return 1/(1+np.exp(-X))
/anaconda3/envs/python36/lib/python3.6/site-packages/statsmodels/discrete/discrete_model.py:

         Current function value: 0.077054
         Iterations: 35
         Function evaluations: 98
         Gradient evaluations: 87


/anaconda3/envs/python36/lib/python3.6/site-packages/statsmodels/base/model.py:508: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  "Check mle_retvals", ConvergenceWarning)


In [54]:
ylabels =['Term novelty ',' Topic novelty' ,'Term_novelty_squared' ,'Topic_novelty_squared', 'Chapters', \
 'Frequent relationship',' Author work count','Age',\
 'Category (Gen)','Category (F/M)',' Category (M/M) ',' Category (Multi)','Category (Other)','Category (Unknown)\
 ','ArchiveWarnings (underage)','ArchiveWarnings (death)',' ArchiveWarnings (violence)','\
 ArchiveWarnings (non-consensual)','ArchiveWarnings (choose not to use)','\
 ArchiveWarnings (no warinigs apply)','Rating (Not rated)', 'Rating (Explicit)','Rating (Mature)',' Rating (Teens)\
 ',' Fandom (DCU) ',' Fandom (Doctor Who) ',' Fandom (Star Wars) ',' Fandom (Arthurian Mythologies)',\
  'Fandom (Supernatural)','Fandom (Haikyuu)',' Fandom (Kuroko no Basuke)',' Fandom (Hamilton/Miranda)\
 ',' Fandom (Dragon Age)',' Fandom (The Walking Dead)',' Fandom (Buffy the Vampire Slayer)','Fandom (Les Miserables)\
 ',' Fandom (Naruto)','Fandom (Tolkien)',' Fandom (Shakespare)','Fandom (Axis Powers Hetalia)',' \
 Fandom (Attack on Titan)',' Fandom (Ms Paint Adventures)',' Fandom (Homestuck)','\
Fandom (Marvel)',' Fandom (Sailor Moon)',' Fandom (One Direction)','Fandom (Sherlock)'][::-1]

In [56]:
hits_model = run_ols(df_all, 'Hits')
hits_coef = list(hits_model.params)[1:-1][::-1]
hits_err = list(hits_model.bse)[1:-1]

/anaconda3/envs/python36/lib/python3.6/site-packages/statsmodels/discrete/discrete_model.py:1674: RuntimeWarning: overflow encountered in exp
  return 1/(1+np.exp(-X))
/anaconda3/envs/python36/lib/python3.6/site-packages/statsmodels/discrete/discrete_model.py:1724: RuntimeWarning: divide by zero encountered in log
  return np.sum(np.log(self.cdf(q*np.dot(X,params))))
/anaconda3/envs/python36/lib/python3.6/site-packages/statsmodels/discrete/discrete_model.py:1674: RuntimeWarning: overflow encountered in exp
  return 1/(1+np.exp(-X))
/anaconda3/envs/python36/lib/python3.6/site-packages/statsmodels/discrete/discrete_model.py:1724: RuntimeWarning: divide by zero encountered in log
  return np.sum(np.log(self.cdf(q*np.dot(X,params))))
/anaconda3/envs/python36/lib/python3.6/site-packages/statsmodels/discrete/discrete_model.py:1674: RuntimeWarning: overflow encountered in exp
  return 1/(1+np.exp(-X))
/anaconda3/envs/python36/lib/python3.6/site-packages/statsmodels/discrete/discrete_model.py:

         Current function value: 0.089942
         Iterations: 35
         Function evaluations: 66
         Gradient evaluations: 58


/anaconda3/envs/python36/lib/python3.6/site-packages/statsmodels/base/model.py:488: HessianInversionWarning: Inverting hessian failed, no bse or cov_params available
  'available', HessianInversionWarning)
/anaconda3/envs/python36/lib/python3.6/site-packages/statsmodels/base/model.py:508: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  "Check mle_retvals", ConvergenceWarning)


PatsyError: Number of rows mismatch between data argument and 'Term_novelty_squared' (697934 versus 1)
    Hits ~ Term_novelty + Topic_novelty + 'Term_novelty_squared' + 'Topic_novelty_squared', + Chapters + Freq_relationship + author_fic_cnt + History + Category_Gen + Category_F_M + Category_M_M + Category_Multi + Category_Other + Category_Unknown  + ArchiveWarnings_underage + ArchiveWarnings_death + ArchiveWarnings_violence + ArchiveWarnings_noncon + ArchiveWarnings_choose_no + ArchiveWarnings_no_apply + Rating_N + Rating_E + Rating_M + Rating_T + Fandom_dcu + Fandom_doctor_who + Fandom_star_wars + Fandom_arthurian +  Fandom_supernatural + Fandom_haikyuu + Fandom_kuroko_no_basuke + Fandom_hamilton_miranda + Fandom_dragon_age + Fandom_the_walking_dead + Fandom_buffy + Fandom_les_miserables  + Fandom_naruto + Fandom_tolkien + Fandom_shakespare + Fandom_hetalia +  Fandom_attack_on_titan + Fandom_ms_paint_adventures + Fandom_homestuck + Fandom_marvel + Fandom_sailor_moon + Fandom_one_dir

In [ ]:
comments_model = run_ols(df_all, 'Comments')
comments_coef = list(comments_model.params)[1:-1][::-1]
comments_err = list(comments_model.bse)[1:-1]

In [ ]:
bookmarks_model = run_ols(df_all, 'Bookmarks')
bookmarks_coef = list(bookmarks_model.params)[1:-1][::-1]
bookmarks_err = list(bookmarks_model.bse)[1:-1]

In [ ]:
print(bookmarks_model.summary())

In [ ]:
fig, ax = plt.subplots(1,4,figsize = (15,15))
ax[0].errorbar(kudos_coef, range(len(kudos_coef)), xerr=kudos_err, fmt='o')
ax[0].set_yticks(range(len(kudos_coef)))
ax[0].set_yticklabels(ylabels)
ax[0].plot([0 for i in range(len(kudos_coef)+2)], range(-1,len(kudos_coef)+1), c='grey', alpha = 0.7)
ax[0].plot(range(-4,3), [22.5 for i in range(-4,3)] ,c='grey', alpha = 0.7)
ax[0].plot(range(-4,3), [26.5 for i in range(-4,3)] ,c='grey', alpha = 0.7)
ax[0].plot(range(-4,3), [32.5 for i in range(-4,3)] ,c='grey', alpha = 0.7)
ax[0].plot(range(-4,3), [38.5 for i in range(-4,3)] ,c='grey', alpha = 0.7)
ax[0].plot(range(-4,3), [42.5 for i in range(-4,3)] ,c='grey', alpha = 0.7)
ax[0].set_ylim(-0.5,44.5)
ax[0].set_xlim(-3,1.5)
ax[0].set_xlabel('Kudos', fontsize=15)
ax[0].set_title('R-squared = 0.260')
for tick in ax[0].yaxis.get_major_ticks():
    tick.label.set_fontsize(13)

ax[1].errorbar(hits_coef, range(len(hits_coef)), xerr=hits_err, fmt='o')
ax[1].set_yticks(range(len(hits_coef)))
# ax[1].set_yticklabels(ylabels)
ax[1].plot([0 for i in range(len(hits_coef)+2)], range(-1,len(hits_coef)+1), c='grey', alpha = 0.7)
ax[1].plot(range(-4,3), [22.5 for i in range(-4,3)] ,c='grey', alpha = 0.7)
ax[1].plot(range(-4,3), [26.5 for i in range(-4,3)] ,c='grey', alpha = 0.7)
ax[1].plot(range(-4,3), [32.5 for i in range(-4,3)] ,c='grey', alpha = 0.7)
ax[1].plot(range(-4,3), [38.5 for i in range(-4,3)] ,c='grey', alpha = 0.7)
ax[1].plot(range(-4,3), [42.5 for i in range(-4,3)] ,c='grey', alpha = 0.7)
ax[1].set_ylim(-0.5,44.5)
ax[1].set_xlim(-3,1.5)
ax[1].set_yticklabels([])
ax[1].set_title('R-squared = 0.154')
ax[1].set_xlabel('Hits', fontsize=15)

ax[2].errorbar(comments_coef, range(len(comments_coef)), xerr=comments_err, fmt='o')
ax[2].set_yticks(range(len(comments_coef)))
ax[2].plot([0 for i in range(len(comments_coef)+2)], range(-1,len(comments_coef)+1), c='grey', alpha = 0.7)
ax[2].plot(range(-4,3), [22.5 for i in range(-4,3)] ,c='grey', alpha = 0.7)
ax[2].plot(range(-4,3), [26.5 for i in range(-4,3)] ,c='grey', alpha = 0.7)
ax[2].plot(range(-4,3), [32.5 for i in range(-4,3)] ,c='grey', alpha = 0.7)
ax[2].plot(range(-4,3), [38.5 for i in range(-4,3)] ,c='grey', alpha = 0.7)
ax[2].plot(range(-4,3), [42.5 for i in range(-4,3)] ,c='grey', alpha = 0.7)
ax[2].set_ylim(-0.5,44.5)
ax[2].set_xlim(-3,1.5)
ax[2].set_yticklabels([])
ax[2].set_title('R-squared = 0.113')
ax[2].set_xlabel('Comments', fontsize=15)


ax[3].errorbar(bookmarks_coef, range(len(bookmarks_coef)), xerr=bookmarks_err, fmt='o')
ax[3].set_yticks(range(len(bookmarks_coef)))
ax[3].plot([0 for i in range(len(bookmarks_coef)+2)], range(-1,len(bookmarks_coef)+1), c='grey', alpha = 0.7)
ax[3].plot(range(-6,3), [22.5 for i in range(-6,3)] ,c='grey', alpha = 0.7)
ax[3].plot(range(-6,3), [26.5 for i in range(-6,3)] ,c='grey', alpha = 0.7)
ax[3].plot(range(-6,3), [32.5 for i in range(-6,3)] ,c='grey', alpha = 0.7)
ax[3].plot(range(-6,3), [38.5 for i in range(-6,3)] ,c='grey', alpha = 0.7)
ax[3].plot(range(-6,3), [42.5 for i in range(-6,3)] ,c='grey', alpha = 0.7)
ax[3].set_ylim(-0.5,44.5)
ax[3].set_xlim(-6,2)
ax[3].set_yticklabels([])
ax[3].set_title('R-squared = 0.238')
ax[3].set_xlabel('Bookmarks', fontsize=15)

plt.tight_layout()
plt.savefig('ols_coefs.pdf', format='pdf')

In [16]:
print(model.summary())

                            OLS Regression Results                            
Dep. Variable:                  Kudos   R-squared:                       0.260
Model:                            OLS   Adj. R-squared:                  0.260
Method:                 Least Squares   F-statistic:                     5441.
Date:                Tue, 24 Jul 2018   Prob (F-statistic):               0.00
Time:                        11:58:23   Log-Likelihood:            -1.3851e+06
No. Observations:              698002   AIC:                         2.770e+06
Df Residuals:                  697956   BIC:                         2.771e+06
Df Model:                          45                                         
Covariance Type:            nonrobust                                         
                                 coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------------------
Intercept           

In [16]:
print(model.summary())

                            OLS Regression Results                            
Dep. Variable:                  Kudos   R-squared:                       0.260
Model:                            OLS   Adj. R-squared:                  0.260
Method:                 Least Squares   F-statistic:                     5441.
Date:                Tue, 24 Jul 2018   Prob (F-statistic):               0.00
Time:                        11:58:23   Log-Likelihood:            -1.3851e+06
No. Observations:              698002   AIC:                         2.770e+06
Df Residuals:                  697956   BIC:                         2.771e+06
Df Model:                          45                                         
Covariance Type:            nonrobust                                         
                                 coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------------------
Intercept           

In [16]:
print(model.summary())

                            OLS Regression Results                            
Dep. Variable:                  Kudos   R-squared:                       0.260
Model:                            OLS   Adj. R-squared:                  0.260
Method:                 Least Squares   F-statistic:                     5441.
Date:                Tue, 24 Jul 2018   Prob (F-statistic):               0.00
Time:                        11:58:23   Log-Likelihood:            -1.3851e+06
No. Observations:              698002   AIC:                         2.770e+06
Df Residuals:                  697956   BIC:                         2.771e+06
Df Model:                          45                                         
Covariance Type:            nonrobust                                         
                                 coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------------------
Intercept           

In [16]:
print(model.summary())

                            OLS Regression Results                            
Dep. Variable:                  Kudos   R-squared:                       0.260
Model:                            OLS   Adj. R-squared:                  0.260
Method:                 Least Squares   F-statistic:                     5441.
Date:                Tue, 24 Jul 2018   Prob (F-statistic):               0.00
Time:                        11:58:23   Log-Likelihood:            -1.3851e+06
No. Observations:              698002   AIC:                         2.770e+06
Df Residuals:                  697956   BIC:                         2.771e+06
Df Model:                          45                                         
Covariance Type:            nonrobust                                         
                                 coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------------------
Intercept           

In [16]:
print(model.summary())

                            OLS Regression Results                            
Dep. Variable:                  Kudos   R-squared:                       0.260
Model:                            OLS   Adj. R-squared:                  0.260
Method:                 Least Squares   F-statistic:                     5441.
Date:                Tue, 24 Jul 2018   Prob (F-statistic):               0.00
Time:                        11:58:23   Log-Likelihood:            -1.3851e+06
No. Observations:              698002   AIC:                         2.770e+06
Df Residuals:                  697956   BIC:                         2.771e+06
Df Model:                          45                                         
Covariance Type:            nonrobust                                         
                                 coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------------------
Intercept           

In [16]:
print(model.summary())

                            OLS Regression Results                            
Dep. Variable:                  Kudos   R-squared:                       0.260
Model:                            OLS   Adj. R-squared:                  0.260
Method:                 Least Squares   F-statistic:                     5441.
Date:                Tue, 24 Jul 2018   Prob (F-statistic):               0.00
Time:                        11:58:23   Log-Likelihood:            -1.3851e+06
No. Observations:              698002   AIC:                         2.770e+06
Df Residuals:                  697956   BIC:                         2.771e+06
Df Model:                          45                                         
Covariance Type:            nonrobust                                         
                                 coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------------------
Intercept           

In [16]:
print(model.summary())

                            OLS Regression Results                            
Dep. Variable:                  Kudos   R-squared:                       0.260
Model:                            OLS   Adj. R-squared:                  0.260
Method:                 Least Squares   F-statistic:                     5441.
Date:                Tue, 24 Jul 2018   Prob (F-statistic):               0.00
Time:                        11:58:23   Log-Likelihood:            -1.3851e+06
No. Observations:              698002   AIC:                         2.770e+06
Df Residuals:                  697956   BIC:                         2.771e+06
Df Model:                          45                                         
Covariance Type:            nonrobust                                         
                                 coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------------------
Intercept           

In [16]:
print(model.summary())

                            OLS Regression Results                            
Dep. Variable:                  Kudos   R-squared:                       0.260
Model:                            OLS   Adj. R-squared:                  0.260
Method:                 Least Squares   F-statistic:                     5441.
Date:                Tue, 24 Jul 2018   Prob (F-statistic):               0.00
Time:                        11:58:23   Log-Likelihood:            -1.3851e+06
No. Observations:              698002   AIC:                         2.770e+06
Df Residuals:                  697956   BIC:                         2.771e+06
Df Model:                          45                                         
Covariance Type:            nonrobust                                         
                                 coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------------------
Intercept           

In [16]:
print(model.summary())

                            OLS Regression Results                            
Dep. Variable:                  Kudos   R-squared:                       0.260
Model:                            OLS   Adj. R-squared:                  0.260
Method:                 Least Squares   F-statistic:                     5441.
Date:                Tue, 24 Jul 2018   Prob (F-statistic):               0.00
Time:                        11:58:23   Log-Likelihood:            -1.3851e+06
No. Observations:              698002   AIC:                         2.770e+06
Df Residuals:                  697956   BIC:                         2.771e+06
Df Model:                          45                                         
Covariance Type:            nonrobust                                         
                                 coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------------------
Intercept           

In [16]:
print(model.summary())

                            OLS Regression Results                            
Dep. Variable:                  Kudos   R-squared:                       0.260
Model:                            OLS   Adj. R-squared:                  0.260
Method:                 Least Squares   F-statistic:                     5441.
Date:                Tue, 24 Jul 2018   Prob (F-statistic):               0.00
Time:                        11:58:23   Log-Likelihood:            -1.3851e+06
No. Observations:              698002   AIC:                         2.770e+06
Df Residuals:                  697956   BIC:                         2.771e+06
Df Model:                          45                                         
Covariance Type:            nonrobust                                         
                                 coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------------------
Intercept           

In [16]:
print(model.summary())

                            OLS Regression Results                            
Dep. Variable:                  Kudos   R-squared:                       0.260
Model:                            OLS   Adj. R-squared:                  0.260
Method:                 Least Squares   F-statistic:                     5441.
Date:                Tue, 24 Jul 2018   Prob (F-statistic):               0.00
Time:                        11:58:23   Log-Likelihood:            -1.3851e+06
No. Observations:              698002   AIC:                         2.770e+06
Df Residuals:                  697956   BIC:                         2.771e+06
Df Model:                          45                                         
Covariance Type:            nonrobust                                         
                                 coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------------------
Intercept           

In [16]:
print(model.summary())

                            OLS Regression Results                            
Dep. Variable:                  Kudos   R-squared:                       0.260
Model:                            OLS   Adj. R-squared:                  0.260
Method:                 Least Squares   F-statistic:                     5441.
Date:                Tue, 24 Jul 2018   Prob (F-statistic):               0.00
Time:                        11:58:23   Log-Likelihood:            -1.3851e+06
No. Observations:              698002   AIC:                         2.770e+06
Df Residuals:                  697956   BIC:                         2.771e+06
Df Model:                          45                                         
Covariance Type:            nonrobust                                         
                                 coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------------------
Intercept           

In [16]:
print(model.summary())

                            OLS Regression Results                            
Dep. Variable:                  Kudos   R-squared:                       0.260
Model:                            OLS   Adj. R-squared:                  0.260
Method:                 Least Squares   F-statistic:                     5441.
Date:                Tue, 24 Jul 2018   Prob (F-statistic):               0.00
Time:                        11:58:23   Log-Likelihood:            -1.3851e+06
No. Observations:              698002   AIC:                         2.770e+06
Df Residuals:                  697956   BIC:                         2.771e+06
Df Model:                          45                                         
Covariance Type:            nonrobust                                         
                                 coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------------------
Intercept           

In [16]:
print(model.summary())

                            OLS Regression Results                            
Dep. Variable:                  Kudos   R-squared:                       0.260
Model:                            OLS   Adj. R-squared:                  0.260
Method:                 Least Squares   F-statistic:                     5441.
Date:                Tue, 24 Jul 2018   Prob (F-statistic):               0.00
Time:                        11:58:23   Log-Likelihood:            -1.3851e+06
No. Observations:              698002   AIC:                         2.770e+06
Df Residuals:                  697956   BIC:                         2.771e+06
Df Model:                          45                                         
Covariance Type:            nonrobust                                         
                                 coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------------------
Intercept           

In [16]:
print(model.summary())

                            OLS Regression Results                            
Dep. Variable:                  Kudos   R-squared:                       0.260
Model:                            OLS   Adj. R-squared:                  0.260
Method:                 Least Squares   F-statistic:                     5441.
Date:                Tue, 24 Jul 2018   Prob (F-statistic):               0.00
Time:                        11:58:23   Log-Likelihood:            -1.3851e+06
No. Observations:              698002   AIC:                         2.770e+06
Df Residuals:                  697956   BIC:                         2.771e+06
Df Model:                          45                                         
Covariance Type:            nonrobust                                         
                                 coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------------------
Intercept           

In [16]:
print(model.summary())

                            OLS Regression Results                            
Dep. Variable:                  Kudos   R-squared:                       0.260
Model:                            OLS   Adj. R-squared:                  0.260
Method:                 Least Squares   F-statistic:                     5441.
Date:                Tue, 24 Jul 2018   Prob (F-statistic):               0.00
Time:                        11:58:23   Log-Likelihood:            -1.3851e+06
No. Observations:              698002   AIC:                         2.770e+06
Df Residuals:                  697956   BIC:                         2.771e+06
Df Model:                          45                                         
Covariance Type:            nonrobust                                         
                                 coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------------------
Intercept           

In [16]:
print(model.summary())

                            OLS Regression Results                            
Dep. Variable:                  Kudos   R-squared:                       0.260
Model:                            OLS   Adj. R-squared:                  0.260
Method:                 Least Squares   F-statistic:                     5441.
Date:                Tue, 24 Jul 2018   Prob (F-statistic):               0.00
Time:                        11:58:23   Log-Likelihood:            -1.3851e+06
No. Observations:              698002   AIC:                         2.770e+06
Df Residuals:                  697956   BIC:                         2.771e+06
Df Model:                          45                                         
Covariance Type:            nonrobust                                         
                                 coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------------------
Intercept           

In [16]:
print(model.summary())

                            OLS Regression Results                            
Dep. Variable:                  Kudos   R-squared:                       0.260
Model:                            OLS   Adj. R-squared:                  0.260
Method:                 Least Squares   F-statistic:                     5441.
Date:                Tue, 24 Jul 2018   Prob (F-statistic):               0.00
Time:                        11:58:23   Log-Likelihood:            -1.3851e+06
No. Observations:              698002   AIC:                         2.770e+06
Df Residuals:                  697956   BIC:                         2.771e+06
Df Model:                          45                                         
Covariance Type:            nonrobust                                         
                                 coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------------------
Intercept           

In [16]:
print(model.summary())

                            OLS Regression Results                            
Dep. Variable:                  Kudos   R-squared:                       0.260
Model:                            OLS   Adj. R-squared:                  0.260
Method:                 Least Squares   F-statistic:                     5441.
Date:                Tue, 24 Jul 2018   Prob (F-statistic):               0.00
Time:                        11:58:23   Log-Likelihood:            -1.3851e+06
No. Observations:              698002   AIC:                         2.770e+06
Df Residuals:                  697956   BIC:                         2.771e+06
Df Model:                          45                                         
Covariance Type:            nonrobust                                         
                                 coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------------------
Intercept           

In [16]:
print(model.summary())

                            OLS Regression Results                            
Dep. Variable:                  Kudos   R-squared:                       0.260
Model:                            OLS   Adj. R-squared:                  0.260
Method:                 Least Squares   F-statistic:                     5441.
Date:                Tue, 24 Jul 2018   Prob (F-statistic):               0.00
Time:                        11:58:23   Log-Likelihood:            -1.3851e+06
No. Observations:              698002   AIC:                         2.770e+06
Df Residuals:                  697956   BIC:                         2.771e+06
Df Model:                          45                                         
Covariance Type:            nonrobust                                         
                                 coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------------------
Intercept           

In [16]:
print(model.summary())

                            OLS Regression Results                            
Dep. Variable:                  Kudos   R-squared:                       0.260
Model:                            OLS   Adj. R-squared:                  0.260
Method:                 Least Squares   F-statistic:                     5441.
Date:                Tue, 24 Jul 2018   Prob (F-statistic):               0.00
Time:                        11:58:23   Log-Likelihood:            -1.3851e+06
No. Observations:              698002   AIC:                         2.770e+06
Df Residuals:                  697956   BIC:                         2.771e+06
Df Model:                          45                                         
Covariance Type:            nonrobust                                         
                                 coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------------------
Intercept           

In [16]:
print(model.summary())

                            OLS Regression Results                            
Dep. Variable:                  Kudos   R-squared:                       0.260
Model:                            OLS   Adj. R-squared:                  0.260
Method:                 Least Squares   F-statistic:                     5441.
Date:                Tue, 24 Jul 2018   Prob (F-statistic):               0.00
Time:                        11:58:23   Log-Likelihood:            -1.3851e+06
No. Observations:              698002   AIC:                         2.770e+06
Df Residuals:                  697956   BIC:                         2.771e+06
Df Model:                          45                                         
Covariance Type:            nonrobust                                         
                                 coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------------------
Intercept           

In [16]:
print(model.summary())

                            OLS Regression Results                            
Dep. Variable:                  Kudos   R-squared:                       0.260
Model:                            OLS   Adj. R-squared:                  0.260
Method:                 Least Squares   F-statistic:                     5441.
Date:                Tue, 24 Jul 2018   Prob (F-statistic):               0.00
Time:                        11:58:23   Log-Likelihood:            -1.3851e+06
No. Observations:              698002   AIC:                         2.770e+06
Df Residuals:                  697956   BIC:                         2.771e+06
Df Model:                          45                                         
Covariance Type:            nonrobust                                         
                                 coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------------------
Intercept           

In [16]:
print(model.summary())

                            OLS Regression Results                            
Dep. Variable:                  Kudos   R-squared:                       0.260
Model:                            OLS   Adj. R-squared:                  0.260
Method:                 Least Squares   F-statistic:                     5441.
Date:                Tue, 24 Jul 2018   Prob (F-statistic):               0.00
Time:                        11:58:23   Log-Likelihood:            -1.3851e+06
No. Observations:              698002   AIC:                         2.770e+06
Df Residuals:                  697956   BIC:                         2.771e+06
Df Model:                          45                                         
Covariance Type:            nonrobust                                         
                                 coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------------------
Intercept           

In [16]:
print(model.summary())

                            OLS Regression Results                            
Dep. Variable:                  Kudos   R-squared:                       0.260
Model:                            OLS   Adj. R-squared:                  0.260
Method:                 Least Squares   F-statistic:                     5441.
Date:                Tue, 24 Jul 2018   Prob (F-statistic):               0.00
Time:                        11:58:23   Log-Likelihood:            -1.3851e+06
No. Observations:              698002   AIC:                         2.770e+06
Df Residuals:                  697956   BIC:                         2.771e+06
Df Model:                          45                                         
Covariance Type:            nonrobust                                         
                                 coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------------------
Intercept           

In [16]:
print(model.summary())

                            OLS Regression Results                            
Dep. Variable:                  Kudos   R-squared:                       0.260
Model:                            OLS   Adj. R-squared:                  0.260
Method:                 Least Squares   F-statistic:                     5441.
Date:                Tue, 24 Jul 2018   Prob (F-statistic):               0.00
Time:                        11:58:23   Log-Likelihood:            -1.3851e+06
No. Observations:              698002   AIC:                         2.770e+06
Df Residuals:                  697956   BIC:                         2.771e+06
Df Model:                          45                                         
Covariance Type:            nonrobust                                         
                                 coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------------------
Intercept           

In [16]:
print(model.summary())

                            OLS Regression Results                            
Dep. Variable:                  Kudos   R-squared:                       0.260
Model:                            OLS   Adj. R-squared:                  0.260
Method:                 Least Squares   F-statistic:                     5441.
Date:                Tue, 24 Jul 2018   Prob (F-statistic):               0.00
Time:                        11:58:23   Log-Likelihood:            -1.3851e+06
No. Observations:              698002   AIC:                         2.770e+06
Df Residuals:                  697956   BIC:                         2.771e+06
Df Model:                          45                                         
Covariance Type:            nonrobust                                         
                                 coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------------------
Intercept           

In [16]:
print(model.summary())

                            OLS Regression Results                            
Dep. Variable:                  Kudos   R-squared:                       0.260
Model:                            OLS   Adj. R-squared:                  0.260
Method:                 Least Squares   F-statistic:                     5441.
Date:                Tue, 24 Jul 2018   Prob (F-statistic):               0.00
Time:                        11:58:23   Log-Likelihood:            -1.3851e+06
No. Observations:              698002   AIC:                         2.770e+06
Df Residuals:                  697956   BIC:                         2.771e+06
Df Model:                          45                                         
Covariance Type:            nonrobust                                         
                                 coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------------------
Intercept           

In [16]:
print(model.summary())

                            OLS Regression Results                            
Dep. Variable:                  Kudos   R-squared:                       0.260
Model:                            OLS   Adj. R-squared:                  0.260
Method:                 Least Squares   F-statistic:                     5441.
Date:                Tue, 24 Jul 2018   Prob (F-statistic):               0.00
Time:                        11:58:23   Log-Likelihood:            -1.3851e+06
No. Observations:              698002   AIC:                         2.770e+06
Df Residuals:                  697956   BIC:                         2.771e+06
Df Model:                          45                                         
Covariance Type:            nonrobust                                         
                                 coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------------------
Intercept           

In [16]:
print(model.summary())

                            OLS Regression Results                            
Dep. Variable:                  Kudos   R-squared:                       0.260
Model:                            OLS   Adj. R-squared:                  0.260
Method:                 Least Squares   F-statistic:                     5441.
Date:                Tue, 24 Jul 2018   Prob (F-statistic):               0.00
Time:                        11:58:23   Log-Likelihood:            -1.3851e+06
No. Observations:              698002   AIC:                         2.770e+06
Df Residuals:                  697956   BIC:                         2.771e+06
Df Model:                          45                                         
Covariance Type:            nonrobust                                         
                                 coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------------------
Intercept           

In [16]:
print(model.summary())

                            OLS Regression Results                            
Dep. Variable:                  Kudos   R-squared:                       0.260
Model:                            OLS   Adj. R-squared:                  0.260
Method:                 Least Squares   F-statistic:                     5441.
Date:                Tue, 24 Jul 2018   Prob (F-statistic):               0.00
Time:                        11:58:23   Log-Likelihood:            -1.3851e+06
No. Observations:              698002   AIC:                         2.770e+06
Df Residuals:                  697956   BIC:                         2.771e+06
Df Model:                          45                                         
Covariance Type:            nonrobust                                         
                                 coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------------------
Intercept           

In [16]:
print(model.summary())

                            OLS Regression Results                            
Dep. Variable:                  Kudos   R-squared:                       0.260
Model:                            OLS   Adj. R-squared:                  0.260
Method:                 Least Squares   F-statistic:                     5441.
Date:                Tue, 24 Jul 2018   Prob (F-statistic):               0.00
Time:                        11:58:23   Log-Likelihood:            -1.3851e+06
No. Observations:              698002   AIC:                         2.770e+06
Df Residuals:                  697956   BIC:                         2.771e+06
Df Model:                          45                                         
Covariance Type:            nonrobust                                         
                                 coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------------------
Intercept           

In [16]:
print(model.summary())

                            OLS Regression Results                            
Dep. Variable:                  Kudos   R-squared:                       0.260
Model:                            OLS   Adj. R-squared:                  0.260
Method:                 Least Squares   F-statistic:                     5441.
Date:                Tue, 24 Jul 2018   Prob (F-statistic):               0.00
Time:                        11:58:23   Log-Likelihood:            -1.3851e+06
No. Observations:              698002   AIC:                         2.770e+06
Df Residuals:                  697956   BIC:                         2.771e+06
Df Model:                          45                                         
Covariance Type:            nonrobust                                         
                                 coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------------------
Intercept           

In [16]:
print(model.summary())

                            OLS Regression Results                            
Dep. Variable:                  Kudos   R-squared:                       0.260
Model:                            OLS   Adj. R-squared:                  0.260
Method:                 Least Squares   F-statistic:                     5441.
Date:                Tue, 24 Jul 2018   Prob (F-statistic):               0.00
Time:                        11:58:23   Log-Likelihood:            -1.3851e+06
No. Observations:              698002   AIC:                         2.770e+06
Df Residuals:                  697956   BIC:                         2.771e+06
Df Model:                          45                                         
Covariance Type:            nonrobust                                         
                                 coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------------------
Intercept           

In [16]:
print(model.summary())

                            OLS Regression Results                            
Dep. Variable:                  Kudos   R-squared:                       0.260
Model:                            OLS   Adj. R-squared:                  0.260
Method:                 Least Squares   F-statistic:                     5441.
Date:                Tue, 24 Jul 2018   Prob (F-statistic):               0.00
Time:                        11:58:23   Log-Likelihood:            -1.3851e+06
No. Observations:              698002   AIC:                         2.770e+06
Df Residuals:                  697956   BIC:                         2.771e+06
Df Model:                          45                                         
Covariance Type:            nonrobust                                         
                                 coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------------------
Intercept           

In [16]:
print(model.summary())

                            OLS Regression Results                            
Dep. Variable:                  Kudos   R-squared:                       0.260
Model:                            OLS   Adj. R-squared:                  0.260
Method:                 Least Squares   F-statistic:                     5441.
Date:                Tue, 24 Jul 2018   Prob (F-statistic):               0.00
Time:                        11:58:23   Log-Likelihood:            -1.3851e+06
No. Observations:              698002   AIC:                         2.770e+06
Df Residuals:                  697956   BIC:                         2.771e+06
Df Model:                          45                                         
Covariance Type:            nonrobust                                         
                                 coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------------------
Intercept           

In [16]:
print(model.summary())

                            OLS Regression Results                            
Dep. Variable:                  Kudos   R-squared:                       0.260
Model:                            OLS   Adj. R-squared:                  0.260
Method:                 Least Squares   F-statistic:                     5441.
Date:                Tue, 24 Jul 2018   Prob (F-statistic):               0.00
Time:                        11:58:23   Log-Likelihood:            -1.3851e+06
No. Observations:              698002   AIC:                         2.770e+06
Df Residuals:                  697956   BIC:                         2.771e+06
Df Model:                          45                                         
Covariance Type:            nonrobust                                         
                                 coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------------------
Intercept           

In [16]:
print(model.summary())

                            OLS Regression Results                            
Dep. Variable:                  Kudos   R-squared:                       0.260
Model:                            OLS   Adj. R-squared:                  0.260
Method:                 Least Squares   F-statistic:                     5441.
Date:                Tue, 24 Jul 2018   Prob (F-statistic):               0.00
Time:                        11:58:23   Log-Likelihood:            -1.3851e+06
No. Observations:              698002   AIC:                         2.770e+06
Df Residuals:                  697956   BIC:                         2.771e+06
Df Model:                          45                                         
Covariance Type:            nonrobust                                         
                                 coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------------------
Intercept           

In [16]:
print(model.summary())

                            OLS Regression Results                            
Dep. Variable:                  Kudos   R-squared:                       0.260
Model:                            OLS   Adj. R-squared:                  0.260
Method:                 Least Squares   F-statistic:                     5441.
Date:                Tue, 24 Jul 2018   Prob (F-statistic):               0.00
Time:                        11:58:23   Log-Likelihood:            -1.3851e+06
No. Observations:              698002   AIC:                         2.770e+06
Df Residuals:                  697956   BIC:                         2.771e+06
Df Model:                          45                                         
Covariance Type:            nonrobust                                         
                                 coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------------------
Intercept           

In [16]:
print(model.summary())

                            OLS Regression Results                            
Dep. Variable:                  Kudos   R-squared:                       0.260
Model:                            OLS   Adj. R-squared:                  0.260
Method:                 Least Squares   F-statistic:                     5441.
Date:                Tue, 24 Jul 2018   Prob (F-statistic):               0.00
Time:                        11:58:23   Log-Likelihood:            -1.3851e+06
No. Observations:              698002   AIC:                         2.770e+06
Df Residuals:                  697956   BIC:                         2.771e+06
Df Model:                          45                                         
Covariance Type:            nonrobust                                         
                                 coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------------------
Intercept           

In [16]:
print(model.summary())

                            OLS Regression Results                            
Dep. Variable:                  Kudos   R-squared:                       0.260
Model:                            OLS   Adj. R-squared:                  0.260
Method:                 Least Squares   F-statistic:                     5441.
Date:                Tue, 24 Jul 2018   Prob (F-statistic):               0.00
Time:                        11:58:23   Log-Likelihood:            -1.3851e+06
No. Observations:              698002   AIC:                         2.770e+06
Df Residuals:                  697956   BIC:                         2.771e+06
Df Model:                          45                                         
Covariance Type:            nonrobust                                         
                                 coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------------------
Intercept           

In [16]:
print(model.summary())

                            OLS Regression Results                            
Dep. Variable:                  Kudos   R-squared:                       0.260
Model:                            OLS   Adj. R-squared:                  0.260
Method:                 Least Squares   F-statistic:                     5441.
Date:                Tue, 24 Jul 2018   Prob (F-statistic):               0.00
Time:                        11:58:23   Log-Likelihood:            -1.3851e+06
No. Observations:              698002   AIC:                         2.770e+06
Df Residuals:                  697956   BIC:                         2.771e+06
Df Model:                          45                                         
Covariance Type:            nonrobust                                         
                                 coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------------------
Intercept           

In [16]:
print(model.summary())

                            OLS Regression Results                            
Dep. Variable:                  Kudos   R-squared:                       0.260
Model:                            OLS   Adj. R-squared:                  0.260
Method:                 Least Squares   F-statistic:                     5441.
Date:                Tue, 24 Jul 2018   Prob (F-statistic):               0.00
Time:                        11:58:23   Log-Likelihood:            -1.3851e+06
No. Observations:              698002   AIC:                         2.770e+06
Df Residuals:                  697956   BIC:                         2.771e+06
Df Model:                          45                                         
Covariance Type:            nonrobust                                         
                                 coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------------------
Intercept           

In [16]:
print(model.summary())

                            OLS Regression Results                            
Dep. Variable:                  Kudos   R-squared:                       0.260
Model:                            OLS   Adj. R-squared:                  0.260
Method:                 Least Squares   F-statistic:                     5441.
Date:                Tue, 24 Jul 2018   Prob (F-statistic):               0.00
Time:                        11:58:23   Log-Likelihood:            -1.3851e+06
No. Observations:              698002   AIC:                         2.770e+06
Df Residuals:                  697956   BIC:                         2.771e+06
Df Model:                          45                                         
Covariance Type:            nonrobust                                         
                                 coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------------------
Intercept           

In [16]:
print(model.summary())

                            OLS Regression Results                            
Dep. Variable:                  Kudos   R-squared:                       0.260
Model:                            OLS   Adj. R-squared:                  0.260
Method:                 Least Squares   F-statistic:                     5441.
Date:                Tue, 24 Jul 2018   Prob (F-statistic):               0.00
Time:                        11:58:23   Log-Likelihood:            -1.3851e+06
No. Observations:              698002   AIC:                         2.770e+06
Df Residuals:                  697956   BIC:                         2.771e+06
Df Model:                          45                                         
Covariance Type:            nonrobust                                         
                                 coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------------------
Intercept           

In [16]:
print(model.summary())

                            OLS Regression Results                            
Dep. Variable:                  Kudos   R-squared:                       0.260
Model:                            OLS   Adj. R-squared:                  0.260
Method:                 Least Squares   F-statistic:                     5441.
Date:                Tue, 24 Jul 2018   Prob (F-statistic):               0.00
Time:                        11:58:23   Log-Likelihood:            -1.3851e+06
No. Observations:              698002   AIC:                         2.770e+06
Df Residuals:                  697956   BIC:                         2.771e+06
Df Model:                          45                                         
Covariance Type:            nonrobust                                         
                                 coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------------------
Intercept           

In [16]:
print(model.summary())

                            OLS Regression Results                            
Dep. Variable:                  Kudos   R-squared:                       0.260
Model:                            OLS   Adj. R-squared:                  0.260
Method:                 Least Squares   F-statistic:                     5441.
Date:                Tue, 24 Jul 2018   Prob (F-statistic):               0.00
Time:                        11:58:23   Log-Likelihood:            -1.3851e+06
No. Observations:              698002   AIC:                         2.770e+06
Df Residuals:                  697956   BIC:                         2.771e+06
Df Model:                          45                                         
Covariance Type:            nonrobust                                         
                                 coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------------------
Intercept           

In [16]:
print(model.summary())

                            OLS Regression Results                            
Dep. Variable:                  Kudos   R-squared:                       0.260
Model:                            OLS   Adj. R-squared:                  0.260
Method:                 Least Squares   F-statistic:                     5441.
Date:                Tue, 24 Jul 2018   Prob (F-statistic):               0.00
Time:                        11:58:23   Log-Likelihood:            -1.3851e+06
No. Observations:              698002   AIC:                         2.770e+06
Df Residuals:                  697956   BIC:                         2.771e+06
Df Model:                          45                                         
Covariance Type:            nonrobust                                         
                                 coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------------------
Intercept           

In [16]:
print(model.summary())

                            OLS Regression Results                            
Dep. Variable:                  Kudos   R-squared:                       0.260
Model:                            OLS   Adj. R-squared:                  0.260
Method:                 Least Squares   F-statistic:                     5441.
Date:                Tue, 24 Jul 2018   Prob (F-statistic):               0.00
Time:                        11:58:23   Log-Likelihood:            -1.3851e+06
No. Observations:              698002   AIC:                         2.770e+06
Df Residuals:                  697956   BIC:                         2.771e+06
Df Model:                          45                                         
Covariance Type:            nonrobust                                         
                                 coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------------------
Intercept           

In [16]:
print(model.summary())

                            OLS Regression Results                            
Dep. Variable:                  Kudos   R-squared:                       0.260
Model:                            OLS   Adj. R-squared:                  0.260
Method:                 Least Squares   F-statistic:                     5441.
Date:                Tue, 24 Jul 2018   Prob (F-statistic):               0.00
Time:                        11:58:23   Log-Likelihood:            -1.3851e+06
No. Observations:              698002   AIC:                         2.770e+06
Df Residuals:                  697956   BIC:                         2.771e+06
Df Model:                          45                                         
Covariance Type:            nonrobust                                         
                                 coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------------------
Intercept           

In [16]:
print(model.summary())

                            OLS Regression Results                            
Dep. Variable:                  Kudos   R-squared:                       0.260
Model:                            OLS   Adj. R-squared:                  0.260
Method:                 Least Squares   F-statistic:                     5441.
Date:                Tue, 24 Jul 2018   Prob (F-statistic):               0.00
Time:                        11:58:23   Log-Likelihood:            -1.3851e+06
No. Observations:              698002   AIC:                         2.770e+06
Df Residuals:                  697956   BIC:                         2.771e+06
Df Model:                          45                                         
Covariance Type:            nonrobust                                         
                                 coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------------------
Intercept           

In [16]:
print(model.summary())

                            OLS Regression Results                            
Dep. Variable:                  Kudos   R-squared:                       0.260
Model:                            OLS   Adj. R-squared:                  0.260
Method:                 Least Squares   F-statistic:                     5441.
Date:                Tue, 24 Jul 2018   Prob (F-statistic):               0.00
Time:                        11:58:23   Log-Likelihood:            -1.3851e+06
No. Observations:              698002   AIC:                         2.770e+06
Df Residuals:                  697956   BIC:                         2.771e+06
Df Model:                          45                                         
Covariance Type:            nonrobust                                         
                                 coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------------------
Intercept           

In [16]:
print(model.summary())

                            OLS Regression Results                            
Dep. Variable:                  Kudos   R-squared:                       0.260
Model:                            OLS   Adj. R-squared:                  0.260
Method:                 Least Squares   F-statistic:                     5441.
Date:                Tue, 24 Jul 2018   Prob (F-statistic):               0.00
Time:                        11:58:23   Log-Likelihood:            -1.3851e+06
No. Observations:              698002   AIC:                         2.770e+06
Df Residuals:                  697956   BIC:                         2.771e+06
Df Model:                          45                                         
Covariance Type:            nonrobust                                         
                                 coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------------------
Intercept           

In [16]:
print(model.summary())

                            OLS Regression Results                            
Dep. Variable:                  Kudos   R-squared:                       0.260
Model:                            OLS   Adj. R-squared:                  0.260
Method:                 Least Squares   F-statistic:                     5441.
Date:                Tue, 24 Jul 2018   Prob (F-statistic):               0.00
Time:                        11:58:23   Log-Likelihood:            -1.3851e+06
No. Observations:              698002   AIC:                         2.770e+06
Df Residuals:                  697956   BIC:                         2.771e+06
Df Model:                          45                                         
Covariance Type:            nonrobust                                         
                                 coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------------------
Intercept           

In [16]:
print(model.summary())

                            OLS Regression Results                            
Dep. Variable:                  Kudos   R-squared:                       0.260
Model:                            OLS   Adj. R-squared:                  0.260
Method:                 Least Squares   F-statistic:                     5441.
Date:                Tue, 24 Jul 2018   Prob (F-statistic):               0.00
Time:                        11:58:23   Log-Likelihood:            -1.3851e+06
No. Observations:              698002   AIC:                         2.770e+06
Df Residuals:                  697956   BIC:                         2.771e+06
Df Model:                          45                                         
Covariance Type:            nonrobust                                         
                                 coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------------------
Intercept           

In [16]:
print(model.summary())

                            OLS Regression Results                            
Dep. Variable:                  Kudos   R-squared:                       0.260
Model:                            OLS   Adj. R-squared:                  0.260
Method:                 Least Squares   F-statistic:                     5441.
Date:                Tue, 24 Jul 2018   Prob (F-statistic):               0.00
Time:                        11:58:23   Log-Likelihood:            -1.3851e+06
No. Observations:              698002   AIC:                         2.770e+06
Df Residuals:                  697956   BIC:                         2.771e+06
Df Model:                          45                                         
Covariance Type:            nonrobust                                         
                                 coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------------------
Intercept           

In [16]:
print(model.summary())

                            OLS Regression Results                            
Dep. Variable:                  Kudos   R-squared:                       0.260
Model:                            OLS   Adj. R-squared:                  0.260
Method:                 Least Squares   F-statistic:                     5441.
Date:                Tue, 24 Jul 2018   Prob (F-statistic):               0.00
Time:                        11:58:23   Log-Likelihood:            -1.3851e+06
No. Observations:              698002   AIC:                         2.770e+06
Df Residuals:                  697956   BIC:                         2.771e+06
Df Model:                          45                                         
Covariance Type:            nonrobust                                         
                                 coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------------------
Intercept           

In [16]:
print(model.summary())

                            OLS Regression Results                            
Dep. Variable:                  Kudos   R-squared:                       0.260
Model:                            OLS   Adj. R-squared:                  0.260
Method:                 Least Squares   F-statistic:                     5441.
Date:                Tue, 24 Jul 2018   Prob (F-statistic):               0.00
Time:                        11:58:23   Log-Likelihood:            -1.3851e+06
No. Observations:              698002   AIC:                         2.770e+06
Df Residuals:                  697956   BIC:                         2.771e+06
Df Model:                          45                                         
Covariance Type:            nonrobust                                         
                                 coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------------------
Intercept           

In [16]:
print(model.summary())

                            OLS Regression Results                            
Dep. Variable:                  Kudos   R-squared:                       0.260
Model:                            OLS   Adj. R-squared:                  0.260
Method:                 Least Squares   F-statistic:                     5441.
Date:                Tue, 24 Jul 2018   Prob (F-statistic):               0.00
Time:                        11:58:23   Log-Likelihood:            -1.3851e+06
No. Observations:              698002   AIC:                         2.770e+06
Df Residuals:                  697956   BIC:                         2.771e+06
Df Model:                          45                                         
Covariance Type:            nonrobust                                         
                                 coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------------------
Intercept           

In [16]:
print(model.summary())

                            OLS Regression Results                            
Dep. Variable:                  Kudos   R-squared:                       0.260
Model:                            OLS   Adj. R-squared:                  0.260
Method:                 Least Squares   F-statistic:                     5441.
Date:                Tue, 24 Jul 2018   Prob (F-statistic):               0.00
Time:                        11:58:23   Log-Likelihood:            -1.3851e+06
No. Observations:              698002   AIC:                         2.770e+06
Df Residuals:                  697956   BIC:                         2.771e+06
Df Model:                          45                                         
Covariance Type:            nonrobust                                         
                                 coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------------------
Intercept           

In [16]:
print(model.summary())

                            OLS Regression Results                            
Dep. Variable:                  Kudos   R-squared:                       0.260
Model:                            OLS   Adj. R-squared:                  0.260
Method:                 Least Squares   F-statistic:                     5441.
Date:                Tue, 24 Jul 2018   Prob (F-statistic):               0.00
Time:                        11:58:23   Log-Likelihood:            -1.3851e+06
No. Observations:              698002   AIC:                         2.770e+06
Df Residuals:                  697956   BIC:                         2.771e+06
Df Model:                          45                                         
Covariance Type:            nonrobust                                         
                                 coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------------------
Intercept           

In [16]:
print(model.summary())

                            OLS Regression Results                            
Dep. Variable:                  Kudos   R-squared:                       0.260
Model:                            OLS   Adj. R-squared:                  0.260
Method:                 Least Squares   F-statistic:                     5441.
Date:                Tue, 24 Jul 2018   Prob (F-statistic):               0.00
Time:                        11:58:23   Log-Likelihood:            -1.3851e+06
No. Observations:              698002   AIC:                         2.770e+06
Df Residuals:                  697956   BIC:                         2.771e+06
Df Model:                          45                                         
Covariance Type:            nonrobust                                         
                                 coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------------------
Intercept           

In [16]:
print(model.summary())

                            OLS Regression Results                            
Dep. Variable:                  Kudos   R-squared:                       0.260
Model:                            OLS   Adj. R-squared:                  0.260
Method:                 Least Squares   F-statistic:                     5441.
Date:                Tue, 24 Jul 2018   Prob (F-statistic):               0.00
Time:                        11:58:23   Log-Likelihood:            -1.3851e+06
No. Observations:              698002   AIC:                         2.770e+06
Df Residuals:                  697956   BIC:                         2.771e+06
Df Model:                          45                                         
Covariance Type:            nonrobust                                         
                                 coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------------------
Intercept           

In [16]:
print(model.summary())

                            OLS Regression Results                            
Dep. Variable:                  Kudos   R-squared:                       0.260
Model:                            OLS   Adj. R-squared:                  0.260
Method:                 Least Squares   F-statistic:                     5441.
Date:                Tue, 24 Jul 2018   Prob (F-statistic):               0.00
Time:                        11:58:23   Log-Likelihood:            -1.3851e+06
No. Observations:              698002   AIC:                         2.770e+06
Df Residuals:                  697956   BIC:                         2.771e+06
Df Model:                          45                                         
Covariance Type:            nonrobust                                         
                                 coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------------------
Intercept           

In [16]:
print(model.summary())

                            OLS Regression Results                            
Dep. Variable:                  Kudos   R-squared:                       0.260
Model:                            OLS   Adj. R-squared:                  0.260
Method:                 Least Squares   F-statistic:                     5441.
Date:                Tue, 24 Jul 2018   Prob (F-statistic):               0.00
Time:                        11:58:23   Log-Likelihood:            -1.3851e+06
No. Observations:              698002   AIC:                         2.770e+06
Df Residuals:                  697956   BIC:                         2.771e+06
Df Model:                          45                                         
Covariance Type:            nonrobust                                         
                                 coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------------------
Intercept           

In [16]:
print(model.summary())

                            OLS Regression Results                            
Dep. Variable:                  Kudos   R-squared:                       0.260
Model:                            OLS   Adj. R-squared:                  0.260
Method:                 Least Squares   F-statistic:                     5441.
Date:                Tue, 24 Jul 2018   Prob (F-statistic):               0.00
Time:                        11:58:23   Log-Likelihood:            -1.3851e+06
No. Observations:              698002   AIC:                         2.770e+06
Df Residuals:                  697956   BIC:                         2.771e+06
Df Model:                          45                                         
Covariance Type:            nonrobust                                         
                                 coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------------------
Intercept           

In [16]:
print(model.summary())

                            OLS Regression Results                            
Dep. Variable:                  Kudos   R-squared:                       0.260
Model:                            OLS   Adj. R-squared:                  0.260
Method:                 Least Squares   F-statistic:                     5441.
Date:                Tue, 24 Jul 2018   Prob (F-statistic):               0.00
Time:                        11:58:23   Log-Likelihood:            -1.3851e+06
No. Observations:              698002   AIC:                         2.770e+06
Df Residuals:                  697956   BIC:                         2.771e+06
Df Model:                          45                                         
Covariance Type:            nonrobust                                         
                                 coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------------------
Intercept           

In [16]:
print(model.summary())

                            OLS Regression Results                            
Dep. Variable:                  Kudos   R-squared:                       0.260
Model:                            OLS   Adj. R-squared:                  0.260
Method:                 Least Squares   F-statistic:                     5441.
Date:                Tue, 24 Jul 2018   Prob (F-statistic):               0.00
Time:                        11:58:23   Log-Likelihood:            -1.3851e+06
No. Observations:              698002   AIC:                         2.770e+06
Df Residuals:                  697956   BIC:                         2.771e+06
Df Model:                          45                                         
Covariance Type:            nonrobust                                         
                                 coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------------------
Intercept           

In [16]:
print(model.summary())

                            OLS Regression Results                            
Dep. Variable:                  Kudos   R-squared:                       0.260
Model:                            OLS   Adj. R-squared:                  0.260
Method:                 Least Squares   F-statistic:                     5441.
Date:                Tue, 24 Jul 2018   Prob (F-statistic):               0.00
Time:                        11:58:23   Log-Likelihood:            -1.3851e+06
No. Observations:              698002   AIC:                         2.770e+06
Df Residuals:                  697956   BIC:                         2.771e+06
Df Model:                          45                                         
Covariance Type:            nonrobust                                         
                                 coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------------------
Intercept           

In [16]:
print(model.summary())

                            OLS Regression Results                            
Dep. Variable:                  Kudos   R-squared:                       0.260
Model:                            OLS   Adj. R-squared:                  0.260
Method:                 Least Squares   F-statistic:                     5441.
Date:                Tue, 24 Jul 2018   Prob (F-statistic):               0.00
Time:                        11:58:23   Log-Likelihood:            -1.3851e+06
No. Observations:              698002   AIC:                         2.770e+06
Df Residuals:                  697956   BIC:                         2.771e+06
Df Model:                          45                                         
Covariance Type:            nonrobust                                         
                                 coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------------------
Intercept           

In [16]:
print(model.summary())

                            OLS Regression Results                            
Dep. Variable:                  Kudos   R-squared:                       0.260
Model:                            OLS   Adj. R-squared:                  0.260
Method:                 Least Squares   F-statistic:                     5441.
Date:                Tue, 24 Jul 2018   Prob (F-statistic):               0.00
Time:                        11:58:23   Log-Likelihood:            -1.3851e+06
No. Observations:              698002   AIC:                         2.770e+06
Df Residuals:                  697956   BIC:                         2.771e+06
Df Model:                          45                                         
Covariance Type:            nonrobust                                         
                                 coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------------------
Intercept           

In [16]:
print(model.summary())

                            OLS Regression Results                            
Dep. Variable:                  Kudos   R-squared:                       0.260
Model:                            OLS   Adj. R-squared:                  0.260
Method:                 Least Squares   F-statistic:                     5441.
Date:                Tue, 24 Jul 2018   Prob (F-statistic):               0.00
Time:                        11:58:23   Log-Likelihood:            -1.3851e+06
No. Observations:              698002   AIC:                         2.770e+06
Df Residuals:                  697956   BIC:                         2.771e+06
Df Model:                          45                                         
Covariance Type:            nonrobust                                         
                                 coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------------------
Intercept           

In [16]:
print(model.summary())

                            OLS Regression Results                            
Dep. Variable:                  Kudos   R-squared:                       0.260
Model:                            OLS   Adj. R-squared:                  0.260
Method:                 Least Squares   F-statistic:                     5441.
Date:                Tue, 24 Jul 2018   Prob (F-statistic):               0.00
Time:                        11:58:23   Log-Likelihood:            -1.3851e+06
No. Observations:              698002   AIC:                         2.770e+06
Df Residuals:                  697956   BIC:                         2.771e+06
Df Model:                          45                                         
Covariance Type:            nonrobust                                         
                                 coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------------------
Intercept           

In [16]:
print(model.summary())

                            OLS Regression Results                            
Dep. Variable:                  Kudos   R-squared:                       0.260
Model:                            OLS   Adj. R-squared:                  0.260
Method:                 Least Squares   F-statistic:                     5441.
Date:                Tue, 24 Jul 2018   Prob (F-statistic):               0.00
Time:                        11:58:23   Log-Likelihood:            -1.3851e+06
No. Observations:              698002   AIC:                         2.770e+06
Df Residuals:                  697956   BIC:                         2.771e+06
Df Model:                          45                                         
Covariance Type:            nonrobust                                         
                                 coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------------------
Intercept           

In [16]:
print(model.summary())

                            OLS Regression Results                            
Dep. Variable:                  Kudos   R-squared:                       0.260
Model:                            OLS   Adj. R-squared:                  0.260
Method:                 Least Squares   F-statistic:                     5441.
Date:                Tue, 24 Jul 2018   Prob (F-statistic):               0.00
Time:                        11:58:23   Log-Likelihood:            -1.3851e+06
No. Observations:              698002   AIC:                         2.770e+06
Df Residuals:                  697956   BIC:                         2.771e+06
Df Model:                          45                                         
Covariance Type:            nonrobust                                         
                                 coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------------------
Intercept           

In [16]:
print(model.summary())

                            OLS Regression Results                            
Dep. Variable:                  Kudos   R-squared:                       0.260
Model:                            OLS   Adj. R-squared:                  0.260
Method:                 Least Squares   F-statistic:                     5441.
Date:                Tue, 24 Jul 2018   Prob (F-statistic):               0.00
Time:                        11:58:23   Log-Likelihood:            -1.3851e+06
No. Observations:              698002   AIC:                         2.770e+06
Df Residuals:                  697956   BIC:                         2.771e+06
Df Model:                          45                                         
Covariance Type:            nonrobust                                         
                                 coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------------------
Intercept           

In [16]:
print(model.summary())

                            OLS Regression Results                            
Dep. Variable:                  Kudos   R-squared:                       0.260
Model:                            OLS   Adj. R-squared:                  0.260
Method:                 Least Squares   F-statistic:                     5441.
Date:                Tue, 24 Jul 2018   Prob (F-statistic):               0.00
Time:                        11:58:23   Log-Likelihood:            -1.3851e+06
No. Observations:              698002   AIC:                         2.770e+06
Df Residuals:                  697956   BIC:                         2.771e+06
Df Model:                          45                                         
Covariance Type:            nonrobust                                         
                                 coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------------------
Intercept           

In [16]:
print(model.summary())

                            OLS Regression Results                            
Dep. Variable:                  Kudos   R-squared:                       0.260
Model:                            OLS   Adj. R-squared:                  0.260
Method:                 Least Squares   F-statistic:                     5441.
Date:                Tue, 24 Jul 2018   Prob (F-statistic):               0.00
Time:                        11:58:23   Log-Likelihood:            -1.3851e+06
No. Observations:              698002   AIC:                         2.770e+06
Df Residuals:                  697956   BIC:                         2.771e+06
Df Model:                          45                                         
Covariance Type:            nonrobust                                         
                                 coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------------------
Intercept           

In [16]:
print(model.summary())

                            OLS Regression Results                            
Dep. Variable:                  Kudos   R-squared:                       0.260
Model:                            OLS   Adj. R-squared:                  0.260
Method:                 Least Squares   F-statistic:                     5441.
Date:                Tue, 24 Jul 2018   Prob (F-statistic):               0.00
Time:                        11:58:23   Log-Likelihood:            -1.3851e+06
No. Observations:              698002   AIC:                         2.770e+06
Df Residuals:                  697956   BIC:                         2.771e+06
Df Model:                          45                                         
Covariance Type:            nonrobust                                         
                                 coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------------------
Intercept           

In [16]:
print(model.summary())

                            OLS Regression Results                            
Dep. Variable:                  Kudos   R-squared:                       0.260
Model:                            OLS   Adj. R-squared:                  0.260
Method:                 Least Squares   F-statistic:                     5441.
Date:                Tue, 24 Jul 2018   Prob (F-statistic):               0.00
Time:                        11:58:23   Log-Likelihood:            -1.3851e+06
No. Observations:              698002   AIC:                         2.770e+06
Df Residuals:                  697956   BIC:                         2.771e+06
Df Model:                          45                                         
Covariance Type:            nonrobust                                         
                                 coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------------------
Intercept           

In [16]:
print(model.summary())

                            OLS Regression Results                            
Dep. Variable:                  Kudos   R-squared:                       0.260
Model:                            OLS   Adj. R-squared:                  0.260
Method:                 Least Squares   F-statistic:                     5441.
Date:                Tue, 24 Jul 2018   Prob (F-statistic):               0.00
Time:                        11:58:23   Log-Likelihood:            -1.3851e+06
No. Observations:              698002   AIC:                         2.770e+06
Df Residuals:                  697956   BIC:                         2.771e+06
Df Model:                          45                                         
Covariance Type:            nonrobust                                         
                                 coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------------------
Intercept           

In [16]:
print(model.summary())

                            OLS Regression Results                            
Dep. Variable:                  Kudos   R-squared:                       0.260
Model:                            OLS   Adj. R-squared:                  0.260
Method:                 Least Squares   F-statistic:                     5441.
Date:                Tue, 24 Jul 2018   Prob (F-statistic):               0.00
Time:                        11:58:23   Log-Likelihood:            -1.3851e+06
No. Observations:              698002   AIC:                         2.770e+06
Df Residuals:                  697956   BIC:                         2.771e+06
Df Model:                          45                                         
Covariance Type:            nonrobust                                         
                                 coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------------------
Intercept           

In [16]:
print(model.summary())

                            OLS Regression Results                            
Dep. Variable:                  Kudos   R-squared:                       0.260
Model:                            OLS   Adj. R-squared:                  0.260
Method:                 Least Squares   F-statistic:                     5441.
Date:                Tue, 24 Jul 2018   Prob (F-statistic):               0.00
Time:                        11:58:23   Log-Likelihood:            -1.3851e+06
No. Observations:              698002   AIC:                         2.770e+06
Df Residuals:                  697956   BIC:                         2.771e+06
Df Model:                          45                                         
Covariance Type:            nonrobust                                         
                                 coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------------------
Intercept           

In [16]:
print(model.summary())

                            OLS Regression Results                            
Dep. Variable:                  Kudos   R-squared:                       0.260
Model:                            OLS   Adj. R-squared:                  0.260
Method:                 Least Squares   F-statistic:                     5441.
Date:                Tue, 24 Jul 2018   Prob (F-statistic):               0.00
Time:                        11:58:23   Log-Likelihood:            -1.3851e+06
No. Observations:              698002   AIC:                         2.770e+06
Df Residuals:                  697956   BIC:                         2.771e+06
Df Model:                          45                                         
Covariance Type:            nonrobust                                         
                                 coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------------------
Intercept           

In [16]:
print(model.summary())

                            OLS Regression Results                            
Dep. Variable:                  Kudos   R-squared:                       0.260
Model:                            OLS   Adj. R-squared:                  0.260
Method:                 Least Squares   F-statistic:                     5441.
Date:                Tue, 24 Jul 2018   Prob (F-statistic):               0.00
Time:                        11:58:23   Log-Likelihood:            -1.3851e+06
No. Observations:              698002   AIC:                         2.770e+06
Df Residuals:                  697956   BIC:                         2.771e+06
Df Model:                          45                                         
Covariance Type:            nonrobust                                         
                                 coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------------------
Intercept           

In [16]:
print(model.summary())

                            OLS Regression Results                            
Dep. Variable:                  Kudos   R-squared:                       0.260
Model:                            OLS   Adj. R-squared:                  0.260
Method:                 Least Squares   F-statistic:                     5441.
Date:                Tue, 24 Jul 2018   Prob (F-statistic):               0.00
Time:                        11:58:23   Log-Likelihood:            -1.3851e+06
No. Observations:              698002   AIC:                         2.770e+06
Df Residuals:                  697956   BIC:                         2.771e+06
Df Model:                          45                                         
Covariance Type:            nonrobust                                         
                                 coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------------------
Intercept           